In [1]:
import re
import pandas as pd
import altair as alt
from pathlib import Path
from functools import reduce

In [2]:
results_path = Path("./dgemm/results")
vortex_results_path = results_path / "VORTEX"
veclib_results_path = results_path / "VECLIB"

In [3]:
def load_results(path: Path, test: str="dgemm") -> pd.DataFrame:
    column_names = ["M", "N", "K", "MFlops"]
    columns_keep = [1, 3, 5, 7]
    df_list = []
    
    for file in path.glob(f"{test}.*.txt"):
        iteration = re.match(r".*\.(\d+)\.txt", str(file))[1]
        df = pd.read_csv(file, sep="\s+", skiprows=2, header=None)[columns_keep]
        df.columns = column_names
        df = df.rename(columns={"MFlops": f"MFlops_{iteration}",
                                "sec": f"sec_{iteration}"})
        df_list.append(df)
        
    results_df = reduce(lambda left, right: pd.merge(left, right, on=column_names[:3]), df_list)
    results_df["label"] = results_df.K
    del results_df["MFlops_1"]
        
    return results_df

def clean_data(df: pd.DataFrame, label: str) -> pd.DataFrame:
    # only retain every 10th test
    df = df[df.K.astype(int).mod(10) == 0]
    mflops_columns = df.filter(axis=1, like="MFlops").columns
    df = df[["label", *mflops_columns]].set_index("label").T.reset_index(drop=True)
    df["blas"] = label
    
    return df

In [4]:
vortex_df = load_results(vortex_results_path)
vortex_df = clean_data(vortex_df, "vortex")

veclib_df = load_results(veclib_results_path)
veclib_df = clean_data(veclib_df, "veclib")

In [17]:
def gen_chart(df: pd.DataFrame):
    df_long = df.melt(id_vars="blas")
    
    line_chart = alt.Chart(df_long).mark_line().encode(
        x=alt.X("label:O", title="M, N, K"),
        y=alt.Y("median(value):Q", title="MFlops"),
        color=alt.Color("blas")
    )
    band_chart = alt.Chart(df_long).mark_errorband(extent='iqr').encode(
        x='label:O',
        y=alt.Y('value', title='MFlops'),
    )
    
    return line_chart + band_chart

vortex_chart = gen_chart(vortex_df)
veclib_chart = gen_chart(veclib_df)

(vortex_chart + veclib_chart).properties(title="dgemm", width=500, height=300)

alt.LayerChart(...)

In [6]:
vortex_neo_results_path = results_path / "VORTEX-NEON1"
vortex_neo_df = load_results(vortex_neo_results_path)
vortex_neo_df = clean_data(vortex_neo_df, "m1-neoverse")

vortex_thunder_results_path = results_path / "VORTEX-THUNDER"
vortex_thunder_df = load_results(vortex_thunder_results_path)
vortex_thunder_df = clean_data(vortex_thunder_df, "m1-thunder")

In [20]:
neo_chart = gen_chart(vortex_neo_df)
thunder_chart = gen_chart(vortex_thunder_df)

(vortex_chart + 
 neo_chart + 
 thunder_chart
).configure_mark(opacity=0.3).properties(title="dgemm", width=600, height=400)

alt.LayerChart(...)

In [9]:
blas_columns = ["vortex/armv8", "m1-neoverse", "m1-thunder"]
charts = []

for col in [70, 100, 120, 160, 200]:
    dgemm_list = [vortex_df[col], vortex_neo_df[col], vortex_thunder_df[col]]

    dgemm_df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dgemm_list)
    dgemm_df.columns = blas_columns

    charts.append(alt.Chart(dgemm_df.melt()).mark_boxplot().encode(
        x=alt.X("variable:O", title="target"),
        y=alt.Y("value:Q", title="MFlops"),
        color=alt.Color("variable", legend=None)
    ).properties(title=f"dgemm: K, M, N = {col}", width=125, height=300))
    
alt.hconcat(*charts)

alt.HConcatChart(...)